In [26]:
Dir.pwd

"/home/osboxes/Desktop/Challenges/JGI_IntroToBioinformatics/Assignment1"

 **Let's first read the files we are working with**

In [1]:
require 'csv'
gene_information = CSV.read("StockDatabaseDataFiles/gene_information.tsv", col_sep: "\t")
p gene_information

[["Gene_ID", "Gene_name", "mutant_phenotype"], ["AT1G69120", "ap1", "meristems replace first and second whorl"], ["AT4G36920", "ap2", "first whorl carpels, second whorl stamens"], ["AT3G54340", "ap3", "second whorl sepals, third whorl carpels"], ["AT1G30950", "ufo", "second whorl sepaloidy, third whorl missing or carpeloid"], ["AT5G20240", "pi", "second whorl sepals, third whorl carpels"]]


[["Gene_ID", "Gene_name", "mutant_phenotype"], ["AT1G69120", "ap1", "meristems replace first and second whorl"], ["AT4G36920", "ap2", "first whorl carpels, second whorl stamens"], ["AT3G54340", "ap3", "second whorl sepals, third whorl carpels"], ["AT1G30950", "ufo", "second whorl sepaloidy, third whorl missing or carpeloid"], ["AT5G20240", "pi", "second whorl sepals, third whorl carpels"]]

In [2]:
#CITATION: Code adapted from
#https://linuxtut.com/how-to-handle-tsv-files-and-csv-files-in-ruby-b8798/

cross_data="StockDatabaseDataFiles/cross_data.tsv"

CSV.foreach(cross_data, col_sep: "\t") do |line|
 p line
end

puts " "

gene_information="StockDatabaseDataFiles/gene_information.tsv"

CSV.foreach(gene_information, col_sep: "\t") do |line|
 p line
end

puts " "

seed_stock_data="StockDatabaseDataFiles/seed_stock_data.tsv"

CSV.foreach(seed_stock_data, col_sep: "\t") do |line|
 p line
end



["Parent1", "Parent2", "F2_Wild", "F2_P1", "F2_P2", "F2_P1P2"]
["A334", "A348", "110", "33", "38", "11"]
["A348", "B3334", "80", "26", "29", "8"]
["B3334", "A51", "99", "36", "34", "12"]
["A51", "B52", "152", "26", "22", "2"]
["B52", "A334", "75", "22", "30", "7"]
 
["Gene_ID", "Gene_name", "mutant_phenotype"]
["AT1G69120", "ap1", "meristems replace first and second whorl"]
["AT4G36920", "ap2", "first whorl carpels, second whorl stamens"]
["AT3G54340", "ap3", "second whorl sepals, third whorl carpels"]
["AT1G30950", "ufo", "second whorl sepaloidy, third whorl missing or carpeloid"]
["AT5G20240", "pi", "second whorl sepals, third whorl carpels"]
 
["Seed_Stock", "Mutant_Gene_ID", "Last_Planted", "Storage", "Grams_Remaining"]
["A334", "AT1G69120", "5/7/2014", "cama2", "28"]
["A348", "AT4G36920", "3/11/2013", "cama25", "12"]
["B3334", "AT3G54340", "1/12/2014", "cama18", "22"]
["A51", "AT1G30950", "9/2/2013", "cama25", "5"]
["B52", "AT5G20240", "8/11/2012", "cama16", "7"]


_________

**TASK 1)** "simulate" planting 7 grams of seeds from each of the records in the seed stock genebank
then you should update the genebank information to show the new quantity of seeds
that remain after a planting. The new state of the genebank
should be printed to a new file, using exactly the same format as the
original file seed_stock_data.tsv

-- if the amount of seed is reduced to zero or less than zero, then
a friendly warning message should appear on the screen. The amount
of seed left in the gene bank is, of course, not LESS than zero

In [37]:
require './SeedStock_object.rb' 

# Specify the path to the input TSV file
input_file = 'StockDatabaseDataFiles/seed_stock_data.tsv'

# Create an array to hold instances of SeedStock
seed_stocks = []

# Read the data from the input file
CSV.foreach(input_file, col_sep: "\t", headers: true) do |row|
  # Create a SeedStock object for each row and store it in the array
  seed_stock = SeedStock.new(row['Seed_Stock'], row['Mutant_Gene_ID'], row['Last_Planted'], row['Storage'], row['Grams_Remaining'])
  seed_stocks << seed_stock
end

# Simulate planting 7 grams of seeds and update the gene bank information
seed_stocks.each { |seed_stock| seed_stock.planting_simulation(7) }

# Specify the path to the output TSV file
output_file = 'new_stock_file.tsv'

# Write the updated data to the output file
CSV.open(output_file, 'w', col_sep: "\t") do |csv|
  # Write the header row
  csv << ['Seed_Stock', 'Mutant_Gene_ID', 'Last_Planted', 'Storage', 'Grams_Remaining']
  # Write the data for each SeedStock object to the file
  seed_stocks.each { |seed_stock| csv << seed_stock.to_csv }
end

puts "Data saved to #{output_file}"

Data saved to new_stock_file.tsv


**TASK 2)** process the information in cross_data.tsv and determine which genes are
genetically-linked. To achieve this, you will have to do a Chi-square test
on the F2 cross data. If you discover genes that are linked, this information
should be added as a property of each of the genes (they are both linked to each
other).

In [38]:
# Gene class 
class Gene
  attr_accessor :gene_id, :gene_name, :mutant_phenotype

  def initialize(gene_id, gene_name, mutant_phenotype)
    @gene_id = gene_id
    @gene_name = gene_name
    @mutant_phenotype = mutant_phenotype
    @linked_genes = [] # Add a property to store linked genes
  end

  def add_linked_gene(gene)
    @linked_genes << gene
    gene.linked_genes << self
  end
end


:add_linked_gene

In [114]:
class HybridCross
  attr_accessor :parent1, :parent2, :f2_wild, :f2_p1, :f2_p2, :f2_p1p2

  def initialize(parent1, parent2, f2_wild, f2_p1, f2_p2, f2_p1p2)
    @parent1 = parent1
    @parent2 = parent2
    @f2_wild = f2_wild.to_i
    @f2_p1 = f2_p1.to_i
    @f2_p2 = f2_p2.to_i
    @f2_p1p2 = f2_p1p2.to_i
  end

  def chi_squared_test
    # Define your observed counts based on the F2 data (numeric data only)
    observed_counts = [f2_wild, f2_p1, f2_p2, f2_p1p2]

    # Calculate the total number of observations for this row
    total_observations = observed_counts.sum

    # Calculate the expected counts for each category based on the 9:3:3:1 ratio
    expected_counts = [
      (total_observations * 9/16.0),
      (total_observations * 3/16.0),
      (total_observations * 3/16.0),
      (total_observations * 1/16.0)
    ]

    # Initialize arrays to store O - E and (O - E)^2 / E values for each category
    o_minus_e = []
    o_minus_e_squared_over_e = []

    # Calculate O - E and (O - E)^2 / E for each category
    (0...observed_counts.size).each do |j|
      observed = observed_counts[j]
      expected = expected_counts[j]
      o_minus_e_value = observed - expected
      o_minus_e_squared_over_e_value = (o_minus_e_value**2).to_f / expected
      o_minus_e << o_minus_e_value
      o_minus_e_squared_over_e << o_minus_e_squared_over_e_value
    end

    # Display a table with Observed (O), Expected (E), O - E, and (O - E)^2 / E for each category
    puts "Observed (O) | Expected (E) | O - E | (O - E)^2 / E"
    (0...observed_counts.size).each do |j|
      puts "#{format('%.2f', observed_counts[j])} | #{format('%.2f', expected_counts[j])} | #{format('%.2f', o_minus_e[j])} | #{format('%.2f', o_minus_e_squared_over_e[j])}"
    end

    # Calculate the chi-squared statistic
    chi_squared = o_minus_e_squared_over_e.sum
    chi_squared
    
  end
end


    
    # Determine degrees of freedom
    #degrees_of_freedom = (row_totals.size - 1) * (column_to


:chi_squared_test

In [120]:
require 'csv'

# Define the HybridCross class as previously shown

# Read data from cross_data.tsv
cross_data = CSV.read('StockDatabaseDataFiles/cross_data.tsv', col_sep: "\t", headers: true)

# Create an array to store instances of the HybridCross class
hybrid_crosses = []

# Iterate through the CSV data and create instances of HybridCross
cross_data.each do |row|
  hybrid_cross = HybridCross.new(
    row['Parent1'],
    row['Parent2'],
    row['F2_Wild'],
    row['F2_P1'],
    row['F2_P2'],
    row['F2_P1P2']
  )
  hybrid_crosses << hybrid_cross
end

# Perform the chi-squared test for each HybridCross instance and print the results
hybrid_crosses.each_with_index do |hybrid_cross, index|
  chi_squared = hybrid_cross.chi_squared_test
  puts "Chi-squared statistic for row #{index + 1}: #{chi_squared}"
end


Observed (O) | Expected (E) | O - E | (O - E)^2 / E
110.00 | 108.00 | 2.00 | 0.04
33.00 | 36.00 | -3.00 | 0.25
38.00 | 36.00 | 2.00 | 0.11
11.00 | 12.00 | -1.00 | 0.08
Chi-squared statistic for row 1: 0.48148148148148145
Observed (O) | Expected (E) | O - E | (O - E)^2 / E
80.00 | 80.44 | -0.44 | 0.00
26.00 | 26.81 | -0.81 | 0.02
29.00 | 26.81 | 2.19 | 0.18
8.00 | 8.94 | -0.94 | 0.10
Chi-squared statistic for row 2: 0.3038073038073038
Observed (O) | Expected (E) | O - E | (O - E)^2 / E
99.00 | 101.81 | -2.81 | 0.08
36.00 | 33.94 | 2.06 | 0.13
34.00 | 33.94 | 0.06 | 0.00
12.00 | 11.31 | 0.69 | 0.04
Chi-squared statistic for row 3: 0.2449355432780847
Observed (O) | Expected (E) | O - E | (O - E)^2 / E
152.00 | 113.62 | 38.38 | 12.96
26.00 | 37.88 | -11.88 | 3.72
22.00 | 37.88 | -15.88 | 6.65
2.00 | 12.62 | -10.62 | 8.94
Chi-squared statistic for row 4: 32.27942794279428
Observed (O) | Expected (E) | O - E | (O - E)^2 / E
75.00 | 75.38 | -0.38 | 0.00
22.00 | 25.12 | -3.12 | 0.39
30.00 | 25

[#<#<Class:0x000055735a69f870>::HybridCross:0x000055735a68c400 @parent1="A334", @parent2="A348", @f2_wild=110, @f2_p1=33, @f2_p2=38, @f2_p1p2=11>, #<#<Class:0x000055735a69f870>::HybridCross:0x000055735a68c090 @parent1="A348", @parent2="B3334", @f2_wild=80, @f2_p1=26, @f2_p2=29, @f2_p1p2=8>, #<#<Class:0x000055735a69f870>::HybridCross:0x000055735a723d28 @parent1="B3334", @parent2="A51", @f2_wild=99, @f2_p1=36, @f2_p2=34, @f2_p1p2=12>, #<#<Class:0x000055735a69f870>::HybridCross:0x000055735a7239e0 @parent1="A51", @parent2="B52", @f2_wild=152, @f2_p1=26, @f2_p2=22, @f2_p1p2=2>, #<#<Class:0x000055735a69f870>::HybridCross:0x000055735a723670 @parent1="B52", @parent2="A334", @f2_wild=75, @f2_p1=22, @f2_p2=30, @f2_p1p2=7>]

In [32]:
class Database
  attr_accessor :genes, :seed_stocks, :hybrid_crosses

  def initialize
    @genes = {}
    @seed_stocks = {}
    @hybrid_crosses = []
  end
end


:initialize